## I'm using time series analysis electric production by ARIMA model

In [ ]:
from pyspark.sql.functions import *
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error
from math import sqrt
import pandas as pd
from pyspark.sql.types import *

#read data 
prod_demand_df = read("TimeSeries_ElectricProduction.ElectricProduction")
#explore data
prod_demand_df.printSchema()
prod_demand_df.show(10)
prod_demand_df.count()


#I found the date is recognized as string 
#I tried to covert Date in string to a date for pandas
prod_demand_df = prod_demand_df.withColumn('Date_date',to_date(prod_demand_df.DATE, 'MM-dd-yyyy')).drop(prod_demand_df.DATE)
prod_demand_df.printSchema()
prod_demand_df.show(10)
#convert spark dataframe to pandas dataframe
pdf = prod_demand_df.toPandas()

# Check the format of 'Date' column
pdf.info()

#spark dataframe with the datatype still recognized as object 
# convert the 'Date' column to datetime format
pdf['Date_date'] = pdf['Date_date'].astype('datetime64[ns]')
 
# Check the format of 'Date' column
pdf.info()


#set index 
pdf.set_index(pd.DatetimeIndex(pdf['Date_date']))
cutoff_date  = "01-01-2017"
before_cutoff = pdf["Date_date"] < cutoff_date
after_cutoff = pdf["Date_date"] >= cutoff_date

train = pdf.loc[before_cutoff]
train.tail(10)
train.info()
print(type(train))
##ts = pd.Series(train['Value'].values, index=train['Date_date'])
model = ARIMA(train['Value'], order=(5,1,0))
model_fit = model.fit()
model_fit.summary()
train['forecast']=model_fit.predict(start=372,end=384,dynamic=True)

train['forecast']
train
test = pdf.loc[after_cutoff]
test 
test['forecast']=model_fit.predict(start=384,end=396,dynamic=True)
test['forecast']
test
#combined train and test dataframes
pdf_with_forecast = pd.concat([train, test])
pdf_with_forecast
result_df = spark.createDataFrame(pdf_with_forecast)
result_df.printSchema()

result_df.show(400)
save(result_df)

### Here is the result in Incorta. The blue line showing original data, and the green line showing predict electric production. 
![this is a pic](https://1.bp.blogspot.com/-GPqDplYRrPQ/YIjv3scG-nI/AAAAAAAAAs0/E7mHXeSJ090v7X0lHXJksBl8eoQWENknwCLcBGAsYHQ/s16000/Screen%2BShot%2B2021-04-27%2Bat%2B10.16.45%2BPM.png)